test

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

model_path = 'microsoft/deberta-v3-small'

tokenizer = AutoTokenizer.from_pretrained(model_path)


/opt/miniconda3/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:


# tokenizer("a ban would only bring problems in gender equality \n We should prohibit women in combat", truncation=True)
train_path = "./group_train.csv"
val_path = "./group_test.csv"
train = load_dataset('csv', data_files=train_path)
val = load_dataset('csv', data_files=val_path)
ds = load_dataset('csv', data_files="./30k.csv", split = 'train')
# train = train.map(preprocess)
# val = val.map(preprocess)
ds = ds.train_test_split(test_size=0.3)
ds

DatasetDict({
    train: Dataset({
        features: ['argument', 'topic', 'set', 'WA', 'MACE-P', 'stance_WA', 'stance_WA_conf'],
        num_rows: 21347
    })
    test: Dataset({
        features: ['argument', 'topic', 'set', 'WA', 'MACE-P', 'stance_WA', 'stance_WA_conf'],
        num_rows: 9150
    })
})

In [13]:
def preprocess(ds):
  text = f"{ds['argument']}.\n{ds['topic']}"
  tokenized_text = tokenizer(text,padding=True , truncation=True)
  tokenized_text['label'] = ds['WA']

  return tokenized_text

train = ds['train']
train = train.map(preprocess)
val = ds['test']
val = val.map(preprocess, batched=True, batch_size=len(val))

Map:   0%|          | 0/21347 [00:00<?, ? examples/s]

Map:   0%|          | 0/9150 [00:00<?, ? examples/s]

ArrowInvalid: Column 7 named input_ids expected length 9150 but got length 293526

[codecarbon INFO @ 00:12:41] Energy consumed for RAM : 0.000950 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:12:41] Energy consumed for all CPUs : 0.000792 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 00:12:41] 0.001742 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:12:56] Energy consumed for RAM : 0.000975 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:12:56] Energy consumed for all CPUs : 0.000813 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 00:12:56] 0.001788 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:13:11] Energy consumed for RAM : 0.001000 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:13:11] Energy consumed for all CPUs : 0.000834 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 00:13:11] 0.001834 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:13:26] Energy consumed for RAM : 0.001025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:13:26] Energy consumed for all CPUs : 0.000854 kWh. Total CPU Power : 5.0 W
[codecarbon

In [7]:
train

Dataset({
    features: ['argument', 'topic', 'set', 'WA', 'MACE-P', 'stance_WA', 'stance_WA_conf', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 21347
})

[codecarbon INFO @ 00:05:56] Energy consumed for RAM : 0.000275 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:05:56] Energy consumed for all CPUs : 0.000229 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 00:05:56] 0.000504 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:06:26] Energy consumed for RAM : 0.000325 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:06:26] Energy consumed for all CPUs : 0.000271 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 00:06:26] 0.000596 kWh of electricity used since the beginning.


In [14]:
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

# def sigmoid(x):
#    return 1/(1 + np.exp(-x))
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):

   predictions, labels = eval_pred
  #  predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(float).reshape(-1))

[codecarbon INFO @ 00:27:41] Energy consumed for RAM : 0.002450 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:27:41] Energy consumed for all CPUs : 0.002042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 00:27:41] 0.004492 kWh of electricity used since the beginning.


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_path)

training_args = TrainingArguments(
    output_dir="./output",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=ds['train'],
   eval_dataset=ds['test'],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

trainer.train()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[codecarbon INFO @ 00:27:52] [setup] RAM Tracking...
[codecarbon INFO @ 00:27:52] [setup] GPU Tracking...
[codecarbon INFO @ 00:27:52] No GPU found.
[codecarbon INFO @ 00:27:52] [setup] CPU Tracking...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism

  0%|          | 0/8007 [00:00<?, ?it/s]

/opt/miniconda3/lib/python3.12/site-packages/datasets/formatting/formatting.py:101: RuntimeWarning: divide by zero encountered in remainder
  return table.fast_gather(key % table.num_rows)


IndexError: list index out of range

[codecarbon INFO @ 00:28:09] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:28:09] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 00:28:09] 0.000046 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:28:11] Energy consumed for RAM : 0.002500 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:28:11] Energy consumed for all CPUs : 0.002084 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 00:28:11] 0.004584 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:28:24] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:28:24] Energy consumed for all CPUs : 0.000042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 00:28:24] 0.000092 kWh of electricity used since the beginning.
[codecarbon INFO @ 00:28:26] Energy consumed for RAM : 0.002525 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 00:28:26] Energy consumed for all CPUs : 0.002105 kWh. Total CPU Power : 5.0 W
[codecarbon